In [1]:
import numpy as np

In [3]:
clip_inference = np.load('checkpoints/clip_inference.npz')

In [14]:
class_ids = clip_inference['class_ids']
similarities = clip_inference['similarities']

In [44]:
k = 10
all_topk_idxs = []
all_mean_scores = []
for i in np.unique(class_ids):
    class_examples = similarities[class_ids == i]
    mean_scores = np.mean(class_examples, axis=0)
    topk_idxs = mean_scores.argsort()[-k:][::-1]
    
    all_topk_idxs.append(topk_idxs)
    all_mean_scores.append(mean_scores)
    
selected_idxs = np.unique(np.concatenate(all_topk_idxs))
selected_mean_scores = np.stack(all_mean_scores)[:, selected_idxs]

In [72]:
x = selected_mean_scores.conj()
# (x - x.min(axis=-1, keepdims=True)) / (x.max(axis=-1, keepdims=True) - x.min(axis=-1, keepdims=True))
x - x.min(axis=-1, keepdims=True)

array([[0.0737 , 0.07983, 0.07983, ..., 0.04883, 0.0675 , 0.062  ],
       [0.05896, 0.0773 , 0.079  , ..., 0.04834, 0.0719 , 0.0702 ],
       [0.06506, 0.0741 , 0.0785 , ..., 0.04236, 0.068  , 0.06116],
       ...,
       [0.06042, 0.08264, 0.05383, ..., 0.04797, 0.0575 , 0.06433],
       [0.1196 , 0.1306 , 0.07227, ..., 0.07715, 0.08496, 0.0786 ],
       [0.07104, 0.0934 , 0.09094, ..., 0.1534 , 0.1261 , 0.1288 ]],
      dtype=float16)

In [66]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scalar = MinMaxScaler().fit(selected_mean_scores.T)
print(scalar.transform(selected_mean_scores.T).T)

[[0.662  0.717  0.717  ... 0.4395 0.6055 0.5566]
 [0.507  0.665  0.6787 ... 0.415  0.618  0.6025]
 [0.585  0.667  0.706  ... 0.3818 0.6123 0.55  ]
 ...
 [0.5625 0.7695 0.502  ... 0.4473 0.535  0.5996]
 [0.8555 0.9355 0.5176 ... 0.552  0.6084 0.5625]
 [0.4316 0.5684 0.5527 ... 0.9336 0.7666 0.783 ]]


In [73]:
import torch
import torch.nn.functional as f

mean_scores_softmax = f.normalize(torch.from_numpy(x - x.min(axis=-1, keepdims=True)), dim=-1)

In [74]:
torch.topk(mean_scores_softmax, 5)

torch.return_types.topk(
values=tensor([[0.0966, 0.0966, 0.0953, 0.0943, 0.0941],
        [0.1001, 0.0986, 0.0967, 0.0951, 0.0948],
        [0.1003, 0.0977, 0.0975, 0.0968, 0.0964],
        [0.1055, 0.1049, 0.1028, 0.1017, 0.1004],
        [0.1075, 0.1038, 0.1005, 0.1002, 0.0981],
        [0.1025, 0.0990, 0.0979, 0.0970, 0.0970],
        [0.1050, 0.0985, 0.0969, 0.0958, 0.0952],
        [0.1035, 0.0993, 0.0948, 0.0938, 0.0930],
        [0.1187, 0.1179, 0.1045, 0.1035, 0.1035],
        [0.0962, 0.0958, 0.0948, 0.0932, 0.0912],
        [0.0941, 0.0924, 0.0908, 0.0905, 0.0903],
        [0.1062, 0.1023, 0.1013, 0.1013, 0.0995],
        [0.0956, 0.0947, 0.0932, 0.0917, 0.0912],
        [0.1108, 0.1095, 0.1078, 0.1048, 0.1035],
        [0.1169, 0.1056, 0.1005, 0.1005, 0.0988],
        [0.1109, 0.1077, 0.0953, 0.0921, 0.0901],
        [0.1140, 0.1073, 0.1059, 0.1048, 0.1039],
        [0.0942, 0.0936, 0.0916, 0.0903, 0.0903],
        [0.1125, 0.1066, 0.1049, 0.1031, 0.1017],
        [0.1126, 0

In [26]:
len(np.unique(np.concatenate(all_topk_idxs)))

232

In [29]:
selected_idxs = np.unique(np.concatenate(all_topk_idxs))

In [30]:
selected_concept_similarities = similarities[:, selected_idxs]

In [31]:
selected_concept_similarities.shape

(5994, 232)

In [33]:
with open('concepts.txt', 'r') as fp:
    concepts = fp.read().splitlines()
selected_concepts = np.array(concepts)[selected_idxs]

In [34]:
selected_concepts

array(['barred with darker brown',
       'barred with darker streaks on a lighter brown background',
       'black beak', 'black bill with yellow tip',
       'black center with white outer feathers',
       'black forehead in males during breeding season', 'black plumage',
       'black primary and secondary feathers visible in flight',
       'black primary feathers', 'black tail with white outer feathers',
       'black throat (males during breeding season)',
       'black throat in males', 'black wings in males, brown in females',
       'black wingtips with white spots',
       'black with blue edges in males', 'black with broad yellow band',
       'black-tipped tail feathers', 'blackish-brown head',
       'blue and black barring on tail', 'blue with white wing bars',
       'blue-gray head', 'blue-gray upper body with fine, white spotting',
       'blue-gray upperparts with breast band',
       'blue-gray wings with darker flight feathers', 'bluish-gray tail',
       'bright b

In [35]:
with open('selected_concepts.txt', 'w') as fp:
    fp.write('\n'.join(selected_concepts))